## Download the data

In [1]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]
###### Do not modify here ###### 


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Get the data shape

In [2]:
print('Train', mnist.train.num_examples, 
      'Validation', mnist.validation.num_examples,
      'Test', mnist.test.num_examples)

print('Train images :', X_train1.shape,
      'Labels :', y_train1.shape)

print('Validation images :', X_valid1.shape,
      'Labels :', y_valid1.shape)

print('Test images :', X_test1.shape,
      'Labels :', y_test1.shape)

Train 55000 Validation 5000 Test 10000
Train images : (28038, 784) Labels : (28038,)
Validation images : (2558, 784) Labels : (2558,)
Test images : (5139, 784) Labels : (5139,)


## One hot-encode y value

In [3]:
y_train_one_hot = np.zeros((len(y_train1), 5))
y_train_one_hot[np.arange(len(y_train1)), y_train1] = 1
y_valid_one_hot = np.zeros((len(y_valid1), 5))
y_valid_one_hot[np.arange(len(y_valid1)), y_valid1] = 1
y_test_one_hot = np.zeros((len(y_test1), 5))
y_test_one_hot[np.arange(len(y_test1)), y_test1] = 1

print(y_train_one_hot[0])

print ("Train Label: ", y_train_one_hot.shape)
print ("Validation Label: ", y_valid_one_hot.shape)
print ("Test Label: ", y_test_one_hot.shape)


[ 0.  0.  0.  1.  0.]
Train Label:  (28038, 5)
Validation Label:  (2558, 5)
Test Label:  (5139, 5)


## Construct the layer

In [4]:
def layer(output_dim, input_dim, inputs, layer, activation=None):
#     W = tf.Variable(tf.random_normal([input_dim, output_dim]))
#     b = tf.get_variable(tf.random_normal([1, output_dim]))
#     W = tf.Variable(tf.truncated_normal(shape = (input_dim, output_dim),mean = 0, stddev = 0.1))
#     b = tf.Variable(tf.zeros(output_dim))

    W = tf.get_variable("W" + layer, shape=[input_dim, output_dim], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable("b" + layer, shape=[1, output_dim], initializer=tf.contrib.layers.xavier_initializer())

    XWb = tf.matmul(inputs, W) + b
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs

### Contruct the input layer

In [5]:
x = tf.placeholder("float", [None, 784])

### Contruct the hidden layer

In [6]:
h1 = layer(output_dim = 128, input_dim = 784, inputs = x, layer="h1", activation = tf.nn.elu)
h2 = layer(output_dim = 128, input_dim = 128, inputs = h1, layer="h2", activation = tf.nn.elu)
h3 = layer(output_dim = 128, input_dim = 128, inputs = h2, layer="h3",activation = tf.nn.elu)
h4 = layer(output_dim = 128, input_dim = 128, inputs = h3, layer="h4",activation = tf.nn.elu)
h5 = layer(output_dim = 128, input_dim = 128, inputs = h4, layer="h5",activation = tf.nn.elu)

### contruct the output layer

In [14]:
y_predict = layer(output_dim = 5, input_dim = 128, inputs = h5, layer="output", activation = None)

ValueError: Variable Woutput already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-4-72a2b6ba7b33>", line 7, in layer
    W = tf.get_variable("W" + layer, shape=[input_dim, output_dim], initializer=tf.contrib.layers.xavier_initializer())
  File "<ipython-input-7-ebb3f3736e49>", line 1, in <module>
    y_predict = layer(output_dim = 5, input_dim = 128, inputs = h5, layer="output", activation = None)
  File "/Users/qq351844/anaconda3/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
y_label = tf.placeholder("float", [None, 5])

In [15]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_predict, labels=y_label))
# spec use tf.nn.sparse_softmax_cross_entropy_with_logits

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001) \
                    .minimize(loss_function)

In [17]:
correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_predict, 1))

In [18]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(15):
        if(i==0):
            batch_x = X_train1[:100]
            batch_y = y_train_one_hot[:100]
        else:
            batch_x = X_train1[i*100:(i+1)*100]
            batch_y = y_train_one_hot[i*100:(i+1)*100]
        sess.run(optimizer, feed_dict={x: batch_x, y_label: batch_y})
    loss, acc = sess.run([loss_function, accuracy], 
                        feed_dict={x: X_valid1, y_label: y_valid_one_hot})
    
    print("Loss: ", loss)
    print("Accurancy: ", acc)

Loss:  0.624158
Accurancy:  0.867866
